# Advanced usage: The `MorphingWorkflow` classes: `MorphingWorkflowGlobal` and `MorphingWorkflowEurope`

This notebook demonstrates the advanced, step-by-step approach to morphing EPW files using the specialized `MorphingWorkflow` classes (`MorphingWorkflowGlobal` and `MorphingWorkflowEurope`).

While the `morph_epw_*` functions are great for direct, one-shot tasks, the workflow classes are designed for complex projects where you need full control over **filename parsing, custom renaming, and process validation**. They enforce a safe, multi-step process that allows you to review and confirm each stage before executing the time-consuming morphing computation.

This notebook will focus on **`MorphingWorkflowGlobal`**, and will show a quick example using **`MorphingWorkflowEurope`** highlighting the differences.

## The Three-Step Workflow

The class is designed to be used as a state machine, guiding you through a logical sequence:

1.  **`map_categories()`**: Analyze the input filenames to extract meaningful data (like city, building type, etc.).
2.  **`configure_and_preview()`**: Define all execution parameters, validate them, and generate a "dry run" plan of how the final files will be named and organized.
3.  **`execute_morphing()`**: Run the final computation, confident that your plan and configuration are correct.

## Using `MorphingWorkflowGlobal`

### General Setup

First, we'll import the necessary class and set up the paths for our files. 

**Important:** You must change the `jar_path` variable to the correct location of the `FutureWeatherGenerator_v3.0.1.jar` file on your system. You must also ensure that the EPW files exist at the specified paths.

In [1]:
import os
import pandas as pd
from pyfwg import MorphingWorkflowGlobal, DEFAULT_GLOBAL_GCMS

# --- Configuration ---
# !!! IMPORTANT: You MUST change this path to the correct location on your PC !!!
jar_path = r"D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar"

# --- Define file paths for the examples ---
pattern_epw_dir = 'epws/w_pattern'
keyword_epw_dir = 'epws/wo_pattern'

### Step 1: Map Categories from Filenames

This is the first and most critical step for organizing your workflow. The `map_categories` method analyzes your source filenames and extracts meaningful data that will be used later for renaming the output files.

#### `map_categories` Parameters

The parameters you can use are:

*   `epw_files` (`List[str]`): **Required.** A list of paths to the EPW files you want to process.
*   `input_filename_pattern` (`Optional[str]`, default: `None`): A Python regex string with **named capture groups** (e.g., `(?P<city>...)`) to extract structured data from filenames.
*   `keyword_mapping` (`Optional[Dict]`, default: `None`): A dictionary of rules. It can be used to normalize values from the pattern or to search the entire filename for keywords to assign categories. The innermost value can be a single string or a list of strings (e.g., `'seville': ['sevilla', 'svq']`).

In [2]:
# Instantiate a workflow object for this example
workflow = MorphingWorkflowGlobal()

Let's have a look at the filenames of the EPWs we are going to work with. First, the set that has a pattern we can define with regex:

In [3]:
# Define the list of files for this specific case
epw_files_with_pattern = [os.path.join(pattern_epw_dir, f) for f in os.listdir(pattern_epw_dir) if f.endswith('.epw')]
print(epw_files_with_pattern)

['epws/w_pattern\\london_uhi-type-2.epw', 'epws/w_pattern\\sevilla_uhi-type-1.epw']


And second, the set that does not have a pattern:

In [4]:
# Define the list of files for this specific case
epw_files_without_pattern = [os.path.join(keyword_epw_dir, f) for f in os.listdir(keyword_epw_dir) if f.endswith('.epw')]
print(epw_files_without_pattern)

['epws/wo_pattern\\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw', 'epws/wo_pattern\\sevilla_in_this_one_the_uhi_is_type-1.epw']


We can also provide a `keyword_mapping` dictionary to translate the extracted raw values (like `sevilla`) into a clean, final format (like `Seville`).

In [5]:
# Define the mapping rules to categorize filenames based on their content.
# This dictionary provides the logic for both keyword searching and normalization.
mapping_rules = {
    # The top-level keys ('city', 'uhi') define the final category names
    # that will be available as placeholders (e.g., {city}) in the output filename.
    'city': {
        # The second-level keys ('Seville', 'London') are the final, clean values
        # that will be assigned to the 'city' category.
        'Seville': ['sevilla', 'SVQ'],  # The keywords to search for (case-insensitive).
        'London': ['london', 'gatwick'] # A list is used for multiple possible keywords.
    },
    'uhi': {
        # For convenience, if there is only one keyword to search for,
        # you can provide it as a single string instead of a list with one item.
        'type-1': 'type-1',
        'type-2': 'type-2'
    }
}

# For example, if a filename contains 'SVQ', pyfwg will assign the value 'Seville'
# to the 'city' category for that file.

#### Example 1.1: Using a Regex Pattern with Normalization

If the filenames follow a consistent pattern, you can use the `input_filename_pattern` argument.

In [6]:
workflow.map_categories(
    epw_files=epw_files_with_pattern,
    # This pattern extracts raw values like 'sevilla' and 'type-1'
    input_filename_pattern=r'(?P<city>.*?)_(?P<uhi>.*)',
    # This dictionary then normalizes them to 'Seville' and 'type-1'. In case of type-1, we want to keep it as it is, so we use the same value.
    keyword_mapping=mapping_rules
)

2025-09-25 09:57:55 - INFO - --- Step 1: Mapping categories from filenames ---
2025-09-25 09:57:55 - INFO - Mapped 'epws/w_pattern\london_uhi-type-2.epw': {'city': 'London', 'uhi': 'uhi-type-2'}
2025-09-25 09:57:55 - INFO - Mapped 'epws/w_pattern\sevilla_uhi-type-1.epw': {'city': 'Seville', 'uhi': 'uhi-type-1'}
2025-09-25 09:57:55 - INFO - Category mapping complete.


The mapped categories are saved to the attribute `.epw_categories` as a dictionary. Let's view it as a DataFrame for better visualization:

In [7]:
pd.DataFrame(workflow.epw_categories)

,epws/w_pattern\london_uhi-type-2.epw,epws/w_pattern\sevilla_uhi-type-1.epw
city,London,Seville
uhi,uhi-type-2,uhi-type-1


#### Example 1.2: Using Keyword-Only Search

If your files are irregularly named, set `input_filename_pattern` to `None`. `pyfwg` will then search for the keywords from your mapping dictionary anywhere in the filename.

In [8]:
workflow.map_categories(
    epw_files=epw_files_without_pattern,
    input_filename_pattern=None,
    keyword_mapping=mapping_rules
)

2025-09-25 09:57:55 - INFO - --- Step 1: Mapping categories from filenames ---
2025-09-25 09:57:55 - INFO - Mapped 'epws/wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw': {'city': 'London', 'uhi': 'type-2'}
2025-09-25 09:57:55 - INFO - Mapped 'epws/wo_pattern\sevilla_in_this_one_the_uhi_is_type-1.epw': {'city': 'Seville', 'uhi': 'type-1'}
2025-09-25 09:57:55 - INFO - Category mapping complete.


Let's view the newly mapped categories:

In [9]:
pd.DataFrame(workflow.epw_categories)

,epws/wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw,epws/wo_pattern\sevilla_in_this_one_the_uhi_is_type-1.epw
city,London,Seville
uhi,type-2,type-1


### Step 2: Configure and Preview the Plan

This is the combined configuration and preview step. Here, you define all the parameters for the Future Weather Generator tool and the output filenames. The method validates everything and then shows you a "dry run" plan of the final results.

#### `configure_and_preview` Parameters

This method has an explicit signature for all possible arguments, providing auto-completion and type hints in your editor.

##### Workflow Parameters
*   `final_output_dir` (`str`): **Required.** The path for the final output files.
*   `output_filename_pattern` (`str`): **Required.** The template for final filenames. Must contain `{ssp}` (or `{rcp}`) and `{year}`. It also can use placeholders from your mapped categories (like `{city}`) and also placeholders for any of the `fwg_` parameters (like `{fwg_interpolation_method_id}`).
*   `scenario_mapping` (`Optional[Dict]`, default: `None`): A dictionary to translate raw scenario names (e.g., `'ssp126'`) into a descriptive format (e.g., `'SSP1-2.6'`).
*   `fwg_jar_path` (`str`): **Required.** Path to the `.jar` file.
*   `run_incomplete_files` (`bool`, default: `False`): If `True`, also processes partially categorized files.
*   `delete_temp_files` (`bool`, default: `True`): If `True`, deletes temporary folders after processing.
*   `temp_base_dir` (`str`, default: `'./morphing_temp_results'`): Base directory for temporary files.
*   `fwg_show_tool_output` (`bool`, default: `False`): If `True`, prints the FWG tool's console output in real-time.
*   `fwg_params` (`Optional[Dict]`, default: `None`): A dictionary for base FWG parameters. Any explicit `fwg_` argument will override this.

##### Future Weather Generator Tool Parameters
*   `fwg_gcms` (`Optional[List[str]]`, default: `None`): **For Global tool only.** A list of GCMs to use.
*   `fwg_rcm_pairs` (`Optional[List[str]]`, default: `None`): **For Europe tool only.** A list of GCM-RCM pairs to use.
*   `fwg_create_ensemble` (`bool`, default: `True`): If `True`, creates an ensemble from the selected models.
*   `fwg_winter_sd_shift` (`float`, default: `0.0`): Winter standard deviation shift.
*   `fwg_summer_sd_shift` (`float`, default: `0.0`): Summer standard deviation shift.
*   `fwg_month_transition_hours` (`int`, default: `72`): Hours for month transition.
*   `fwg_use_multithreading` (`bool`, default: `True`): Use multithreading.
*   `fwg_interpolation_method_id` (`int`, default: `0`): Interpolation method ID.
*   `fwg_limit_variables` (`bool`, default: `True`): Limit variables to physical bounds.
*   `fwg_solar_hour_adjustment` (`int`, default: `1`): Solar hour adjustment option.
*   `fwg_diffuse_irradiation_model` (`int`, default: `1`): Diffuse irradiation model option.
*   `fwg_add_uhi` (`bool`, default: `True`): Add UHI effect.
*   `fwg_epw_original_lcz` (`int`, default: `14`): Original EPW LCZ.
*   `fwg_target_uhi_lcz` (`int`, default: `1`): Target UHI LCZ.

In [10]:
# We will continue with the 'workflow' object, which now contains the keyword-mapped files.
workflow.configure_and_preview(
    final_output_dir='./final_results_workflow_global',
    # The placeholders {city} and {uhi} match the mapping rule keys
    output_filename_pattern='{city}_{uhi}_{ssp}_{year}_interp-{fwg_interpolation_method_id}',
    # The {ssp} placeholder will be populated from this mapping
    # For instance, the SSP scenario will be shown as 'SSP2-4.5' in the output filename instead of 'ssp245'.
    scenario_mapping={'ssp245': 'SSP2-4.5', 'ssp585': 'SSP5-8.5'},
    
    # --- FWG Configuration ---
    fwg_jar_path=jar_path,
    fwg_gcms=['BCC_CSM2_MR'], # Use just one GCM for a quick test
    fwg_interpolation_method_id=2, # This value will appear in the filename
    fwg_epw_original_lcz=2,         # Use a validated LCZ from the check above
    fwg_target_uhi_lcz=3,          # Use a validated LCZ from the check above
    fwg_show_tool_output=True,
    delete_temp_files=False # Set to False for debugging
)

2025-09-25 09:57:55 - INFO - --- Step 2: Configuring and Previewing Morphing Plan ---



          MORPHING CONFIGURATION & PREVIEW
  - FWG JAR Path: D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar
  - Final Output Directory: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_global
  - EPWs to be Morphed (2 files):
    - GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw
    - sevilla_in_this_one_the_uhi_is_type-1.epw

  For input file: GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw
    -> Generated 'ssp126_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_global\London_type-2_ssp126_2050_interp-2.epw
    -> Generated 'ssp245_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_global\London_type-2_SSP2-4.5_2050_interp-2.epw
    -> Generated 'ssp370_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_global\London_type-2_ssp370_2050_interp-2.epw
    -> Generated 'ssp585_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_result

You can also inspect the rename plan just shown above in the attribute `.rename_plan`

In [11]:
workflow.rename_plan

{'epws/wo_pattern\\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw': {'ssp126_2050.epw': './final_results_workflow_global\\London_type-2_ssp126_2050_interp-2.epw',
  'ssp245_2050.epw': './final_results_workflow_global\\London_type-2_SSP2-4.5_2050_interp-2.epw',
  'ssp370_2050.epw': './final_results_workflow_global\\London_type-2_ssp370_2050_interp-2.epw',
  'ssp585_2050.epw': './final_results_workflow_global\\London_type-2_SSP5-8.5_2050_interp-2.epw',
  'ssp126_2080.epw': './final_results_workflow_global\\London_type-2_ssp126_2080_interp-2.epw',
  'ssp245_2080.epw': './final_results_workflow_global\\London_type-2_SSP2-4.5_2080_interp-2.epw',
  'ssp370_2080.epw': './final_results_workflow_global\\London_type-2_ssp370_2080_interp-2.epw',
  'ssp585_2080.epw': './final_results_workflow_global\\London_type-2_SSP5-8.5_2080_interp-2.epw'},
 'epws/wo_pattern\\sevilla_in_this_one_the_uhi_is_type-1.epw': {'ssp126_2050.epw': './final_results_workflow_global\\Seville_type-1_ssp126_2050_interp-2.epw'

The inputs you have defined as well as the default values automatically set for the arguments not specified can be inspected in the attribute `.inputs` prior to running the morphing workflow.

In [12]:
workflow.inputs

{'epw_files': ['epws/wo_pattern\\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw',
  'epws/wo_pattern\\sevilla_in_this_one_the_uhi_is_type-1.epw'],
 'final_output_dir': './final_results_workflow_global',
 'output_filename_pattern': '{city}_{uhi}_{ssp}_{year}_interp-{fwg_interpolation_method_id}',
 'scenario_mapping': {'ssp245': 'SSP2-4.5', 'ssp585': 'SSP5-8.5'},
 'fwg_jar_path': 'D:\\OneDrive - Universidad de Cádiz (uca.es)\\Programas\\FutureWeatherGenerator_v3.0.1.jar',
 'run_incomplete_files': False,
 'delete_temp_files': False,
 'temp_base_dir': './morphing_temp_results',
 'show_tool_output': True,
 'fwg_params': {'gcms': ['BCC_CSM2_MR'],
  'create_ensemble': True,
  'winter_sd_shift': 0.0,
  'summer_sd_shift': 0.0,
  'month_transition_hours': 72,
  'use_multithreading': True,
  'interpolation_method_id': 2,
  'limit_variables': True,
  'solar_hour_adjustment': 1,
  'diffuse_irradiation_model': 1,
  'add_uhi': True,
  'epw_original_lcz': 2,
  'target_uhi_lcz': 3},
 'fwg_params_formatt

The attribute `.inputs` is a dictionary, which contains among others the arguments used in the FWG tool, that is, the values that are going to be passed to the CMD prompt. You can inspect these in the dictionary nested in key `fwg_params`.

In [13]:
workflow.inputs['fwg_params']

{'gcms': ['BCC_CSM2_MR'],
 'create_ensemble': True,
 'winter_sd_shift': 0.0,
 'summer_sd_shift': 0.0,
 'month_transition_hours': 72,
 'use_multithreading': True,
 'interpolation_method_id': 2,
 'limit_variables': True,
 'solar_hour_adjustment': 1,
 'diffuse_irradiation_model': 1,
 'add_uhi': True,
 'epw_original_lcz': 2,
 'target_uhi_lcz': 3}

### Step 3: Execute the Morphing Process

This is the final step. The `execute_morphing` method takes **no arguments**. It acts as the "Go" button, running the entire process based on the configuration from the previous step.

In [14]:
workflow.execute_morphing()

2025-09-25 09:57:55 - INFO - --- Step 4: Executing morphing workflow ---
2025-09-25 09:57:55 - INFO - Validating LCZ availability for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw...
2025-09-25 09:57:55 - INFO - Checking LCZ pair (Original: 2, Target: 3) availability for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw...
2025-09-25 09:57:55 - INFO - --- Applying UHI effect to GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw ---
2025-09-25 09:57:55 - INFO - Executing command: java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar" futureweathergenerator.UHI_Morph D:\Python\pyfwg\pyfwg\tutorials\epws\wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw C:\Users\sanga\AppData\Local\Temp\tmpglj33sco/ true 2:3
2025-09-25 09:58:01 - INFO - UHI effect applied successfully.
2025-09-25 09:58:01 - INFO - LCZ pair (Original: 2, Target: 3) is available.
2025-09-25 09:58:01 - INFO - Copied input file to temporary directory: ./morphing_temp_results\GBR_Londo


-------------------- Executing FWG for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw --------------------
  Full Command (for reference): java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar" futureweathergenerator.Morph D:\Python\pyfwg\pyfwg\tutorials\epws\wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw BCC_CSM2_MR 1 0.0:0.0 72 D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results\GBR_London.Gatwick.037760_IWEC_uhi_type-2/ true 2 true 1 1 1:2:3
  --- FWG Real-time Output ---


2025-09-25 09:58:56 - INFO - Processing generated files in: ./morphing_temp_results\GBR_London.Gatwick.037760_IWEC_uhi_type-2
2025-09-25 09:58:56 - INFO - Skipping auxiliary file: 'GBR_-_LONDON-GATWICK_BCC_CSM2_MR_ssp126_2050_GridPointVariables.csv'
2025-09-25 09:58:56 - INFO - Skipping auxiliary file: 'GBR_-_LONDON-GATWICK_BCC_CSM2_MR_ssp126_2080_GridPointVariables.csv'
2025-09-25 09:58:56 - INFO - Skipping auxiliary file: 'GBR_-_LONDON-GATWICK_BCC_CSM2_MR_ssp245_2050_GridPointVariables.csv'
2025-09-25 09:58:56 - INFO - Skipping auxiliary file: 'GBR_-_LONDON-GATWICK_BCC_CSM2_MR_ssp245_2080_GridPointVariables.csv'
2025-09-25 09:58:56 - INFO - Skipping auxiliary file: 'GBR_-_LONDON-GATWICK_BCC_CSM2_MR_ssp370_2050_GridPointVariables.csv'
2025-09-25 09:58:56 - INFO - Skipping auxiliary file: 'GBR_-_LONDON-GATWICK_BCC_CSM2_MR_ssp370_2080_GridPointVariables.csv'
2025-09-25 09:58:56 - INFO - Skipping auxiliary file: 'GBR_-_LONDON-GATWICK_BCC_CSM2_MR_ssp585_2050_GridPointVariables.csv'
2025-0

  --- End of FWG Output ---


2025-09-25 09:59:00 - INFO - UHI effect applied successfully.
2025-09-25 09:59:00 - INFO - LCZ pair (Original: 2, Target: 3) is available.
2025-09-25 09:59:00 - INFO - Copied input file to temporary directory: ./morphing_temp_results\sevilla_in_this_one_the_uhi_is_type-1\sevilla_in_this_one_the_uhi_is_type-1.epw



-------------------- Executing FWG for sevilla_in_this_one_the_uhi_is_type-1.epw --------------------
  Full Command (for reference): java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_v3.0.1.jar" futureweathergenerator.Morph D:\Python\pyfwg\pyfwg\tutorials\epws\wo_pattern\sevilla_in_this_one_the_uhi_is_type-1.epw BCC_CSM2_MR 1 0.0:0.0 72 D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results\sevilla_in_this_one_the_uhi_is_type-1/ true 2 true 1 1 1:2:3
  --- FWG Real-time Output ---


2025-09-25 09:59:46 - INFO - Processing generated files in: ./morphing_temp_results\sevilla_in_this_one_the_uhi_is_type-1
2025-09-25 09:59:46 - INFO - Skipping auxiliary file: 'ESP_-_SEVILLA_BCC_CSM2_MR_ssp126_2050_GridPointVariables.csv'
2025-09-25 09:59:46 - INFO - Skipping auxiliary file: 'ESP_-_SEVILLA_BCC_CSM2_MR_ssp126_2080_GridPointVariables.csv'
2025-09-25 09:59:46 - INFO - Skipping auxiliary file: 'ESP_-_SEVILLA_BCC_CSM2_MR_ssp245_2050_GridPointVariables.csv'
2025-09-25 09:59:46 - INFO - Skipping auxiliary file: 'ESP_-_SEVILLA_BCC_CSM2_MR_ssp245_2080_GridPointVariables.csv'
2025-09-25 09:59:46 - INFO - Skipping auxiliary file: 'ESP_-_SEVILLA_BCC_CSM2_MR_ssp370_2050_GridPointVariables.csv'
2025-09-25 09:59:46 - INFO - Skipping auxiliary file: 'ESP_-_SEVILLA_BCC_CSM2_MR_ssp370_2080_GridPointVariables.csv'
2025-09-25 09:59:46 - INFO - Skipping auxiliary file: 'ESP_-_SEVILLA_BCC_CSM2_MR_ssp585_2050_GridPointVariables.csv'
2025-09-25 09:59:46 - INFO - Skipping auxiliary file: 'ESP_

  --- End of FWG Output ---


Let's take a look at the files we have created.

In [15]:
new_files = [i for i in os.listdir('./final_results_workflow_global')]
new_files

['London_type-2_ssp126_2050_interp-2.epw',
 'London_type-2_ssp126_2050_interp-2.stat',
 'London_type-2_ssp126_2080_interp-2.epw',
 'London_type-2_ssp126_2080_interp-2.stat',
 'London_type-2_SSP2-4.5_2050_interp-2.epw',
 'London_type-2_SSP2-4.5_2050_interp-2.stat',
 'London_type-2_SSP2-4.5_2080_interp-2.epw',
 'London_type-2_SSP2-4.5_2080_interp-2.stat',
 'London_type-2_ssp370_2050_interp-2.epw',
 'London_type-2_ssp370_2050_interp-2.stat',
 'London_type-2_ssp370_2080_interp-2.epw',
 'London_type-2_ssp370_2080_interp-2.stat',
 'London_type-2_SSP5-8.5_2050_interp-2.epw',
 'London_type-2_SSP5-8.5_2050_interp-2.stat',
 'London_type-2_SSP5-8.5_2080_interp-2.epw',
 'London_type-2_SSP5-8.5_2080_interp-2.stat',
 'Seville_type-1_ssp126_2050_interp-2.epw',
 'Seville_type-1_ssp126_2050_interp-2.stat',
 'Seville_type-1_ssp126_2080_interp-2.epw',
 'Seville_type-1_ssp126_2080_interp-2.stat',
 'Seville_type-1_SSP2-4.5_2050_interp-2.epw',
 'Seville_type-1_SSP2-4.5_2050_interp-2.stat',
 'Seville_type-1_

Let's delete the files we just generated.

In [16]:
import shutil
shutil.rmtree('final_results_workflow_global')

## Using `MorphingWorkflowEurope`

### Differences compared to `MorphingWorkflowGlobal`
>**FYI**: the usage of `MorphingWorkflowEurope` is very similar to the global version. The only differences are:

>* Instead of argument `fwg_gcms`, you need to use `fwg_rcm_pairs`.
>* Instead of variable `DEFAULT_GLOBAL_GCMS`, the available models are in variable `DEFAULT_EUROPE_RCMS`. Therefore, the models you can use in `fwg_rcm_pairs` are those included in `DEFAULT_EUROPE_RCMS`.
>* Future scenarios are RCP instead of SSP. Therefore, in `.configure_and_preview`'s' `scenario_mapping` argument, the dictionary keys must be `'rcp26'`, `'rcp45'` and `'rcp85'`. Also, instead of placeholder `{ssp}`, you must use placeholder **`{rcp}`**
>* You have to remember using the `jar_path` to the FutureWeatherGenerator_Europe_vx.x.x.jar file.

### Example of use

Since we have already explain the details in the `MorphingWorkflowGlobal` class and highlighted the differences, we are going to show the usage of `MorphingWorkflowEurope` in the following cell.

In [17]:
import os
import pandas as pd
from pyfwg import MorphingWorkflowEurope, DEFAULT_EUROPE_RCMS

# --- Configuration ---
# !!! IMPORTANT: You MUST change this path to the correct location on your PC !!!
# Also, remember to define the path to the FutureWeatherGenerator_Europe_vx.x.x.jar file
# if you are going to instantiate a MorphingWorkflowEurope class.
jar_path = r"D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v1.0.1.jar"

# --- Define file paths for the examples ---
pattern_epw_dir = 'epws/w_pattern'
keyword_epw_dir = 'epws/wo_pattern'

# Instantiate a workflow object for this example
workflow = MorphingWorkflowEurope()

# Define the list of files for this specific case
epw_files_without_pattern = [os.path.join(keyword_epw_dir, f) for f in os.listdir(keyword_epw_dir) if f.endswith('.epw')]

# Define the mapping rules to categorize filenames based on their content.
# This dictionary provides the logic for both keyword searching and normalization.
mapping_rules = {
    # The top-level keys ('city', 'uhi') define the final category names
    # that will be available as placeholders (e.g., {city}) in the output filename.
    'city': {
        # The second-level keys ('Seville', 'London') are the final, clean values
        # that will be assigned to the 'city' category.
        'Seville': ['sevilla', 'SVQ'],  # The keywords to search for (case-insensitive).
        'London': ['london', 'gatwick'] # A list is used for multiple possible keywords.
    },
    'uhi': {
        # For convenience, if there is only one keyword to search for,
        # you can provide it as a single string instead of a list with one item.
        'type-1': 'type-1',
        'type-2': 'type-2'
    }
}
# For example, if a filename contains 'SVQ', pyfwg will assign the value 'Seville'
# to the 'city' category for that file.

# Let's map the files to the categories.
workflow.map_categories(
    epw_files=epw_files_without_pattern,
    input_filename_pattern=None,
    keyword_mapping=mapping_rules
)

# We will continue with the 'workflow' object, which now contains the keyword-mapped files.
workflow.configure_and_preview(
    final_output_dir='./final_results_workflow_europe',
    # The placeholders {city} and {uhi} match the mapping rule keys
    output_filename_pattern='{city}_{uhi}_{rcp}_{year}_interp-{fwg_interpolation_method_id}',
    # The {ssp} placeholder will be populated from this mapping
    # For instance, the SSP scenario will be shown as 'SSP2-4.5' in the output filename instead of 'ssp245'.
    scenario_mapping={'rcp26': 'RCP-2.6'},

    # --- FWG Configuration ---
    fwg_jar_path=jar_path,
    # Important: remember to use fwg_rcm_pairs in the MorphingWorkflowEurope instead of fwg_gcms!
    fwg_rcm_pairs=[list(DEFAULT_EUROPE_RCMS)[0]],  # Let's use the first RCM of the list for a quick test. Remember it must be a list (even if it contains just 1 item)
    fwg_interpolation_method_id=2,  # This value will appear in the filename
    fwg_epw_original_lcz=2,  # Use a validated LCZ from the check above
    fwg_target_uhi_lcz=3,  # Use a validated LCZ from the check above
    fwg_show_tool_output=True,
    delete_temp_files=False  # Set to False for debugging
)

# Execute the morphing process
workflow.execute_morphing()

2025-09-25 09:59:47 - INFO - --- Step 1: Mapping categories from filenames ---
2025-09-25 09:59:47 - INFO - Mapped 'epws/wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw': {'city': 'London', 'uhi': 'type-2'}
2025-09-25 09:59:47 - INFO - Mapped 'epws/wo_pattern\sevilla_in_this_one_the_uhi_is_type-1.epw': {'city': 'Seville', 'uhi': 'type-1'}
2025-09-25 09:59:47 - INFO - Category mapping complete.
2025-09-25 09:59:47 - INFO - --- Step 2: Configuring and Previewing Morphing Plan ---
2025-09-25 09:59:47 - INFO - --- Step 4: Executing morphing workflow ---
2025-09-25 09:59:47 - INFO - Validating LCZ availability for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw...
2025-09-25 09:59:47 - INFO - Checking LCZ pair (Original: 2, Target: 3) availability for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw...
2025-09-25 09:59:47 - INFO - --- Applying UHI effect to GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw ---
2025-09-25 09:59:47 - INFO - Executing command: java -cp "D:\OneDrive - Universid


          MORPHING CONFIGURATION & PREVIEW
  - FWG JAR Path: D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v1.0.1.jar
  - Final Output Directory: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_europe
  - EPWs to be Morphed (2 files):
    - GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw
    - sevilla_in_this_one_the_uhi_is_type-1.epw

  For input file: GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw
    -> Generated 'rcp26_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_europe\London_type-2_RCP-2.6_2050_interp-2.epw
    -> Generated 'rcp45_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_europe\London_type-2_rcp45_2050_interp-2.epw
    -> Generated 'rcp85_2050.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_results_workflow_europe\London_type-2_rcp85_2050_interp-2.epw
    -> Generated 'rcp26_2080.epw' will be moved to: D:\Python\pyfwg\pyfwg\tutorials\final_result

2025-09-25 09:59:51 - INFO - UHI effect applied successfully.
2025-09-25 09:59:51 - INFO - LCZ pair (Original: 2, Target: 3) is available.
2025-09-25 09:59:51 - INFO - Copied input file to temporary directory: ./morphing_temp_results_europe\GBR_London.Gatwick.037760_IWEC_uhi_type-2\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw



-------------------- Executing FWG for GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw --------------------
  Full Command (for reference): java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v1.0.1.jar" futureweathergenerator_europe.Morph D:\Python\pyfwg\pyfwg\tutorials\epws\wo_pattern\GBR_London.Gatwick.037760_IWEC_uhi_type-2.epw ICHEC_EC_EARTH_SMHI_RCA4 1 0.0:0.0 72 D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results_europe\GBR_London.Gatwick.037760_IWEC_uhi_type-2/ true 2 true 1 1 1:2:3
  --- FWG Real-time Output ---


2025-09-25 10:00:50 - INFO - Processing generated files in: ./morphing_temp_results_europe\GBR_London.Gatwick.037760_IWEC_uhi_type-2
2025-09-25 10:00:50 - INFO - Moving './morphing_temp_results_europe\GBR_London.Gatwick.037760_IWEC_uhi_type-2\GBR_-_LONDON-GATWICK_Ensemble_rcp26_2050.epw' to './final_results_workflow_europe\London_type-2_RCP-2.6_2050_interp-2.epw'
2025-09-25 10:00:50 - INFO - Moving './morphing_temp_results_europe\GBR_London.Gatwick.037760_IWEC_uhi_type-2\GBR_-_LONDON-GATWICK_Ensemble_rcp26_2050.stat' to './final_results_workflow_europe\London_type-2_RCP-2.6_2050_interp-2.stat'
2025-09-25 10:00:50 - INFO - Skipping auxiliary file: 'GBR_-_LONDON-GATWICK_Ensemble_rcp26_2050_ErrorsAndWarnings.log'
2025-09-25 10:00:50 - INFO - Skipping auxiliary file: 'GBR_-_LONDON-GATWICK_Ensemble_rcp26_2050_GridPointVariables.csv'
2025-09-25 10:00:50 - INFO - Skipping auxiliary file: 'GBR_-_LONDON-GATWICK_Ensemble_rcp26_2050_MorphedEPWsComparison.csv'
2025-09-25 10:00:50 - INFO - Moving '

  --- End of FWG Output ---


2025-09-25 10:00:56 - INFO - UHI effect applied successfully.
2025-09-25 10:00:56 - INFO - LCZ pair (Original: 2, Target: 3) is available.
2025-09-25 10:00:56 - INFO - Copied input file to temporary directory: ./morphing_temp_results_europe\sevilla_in_this_one_the_uhi_is_type-1\sevilla_in_this_one_the_uhi_is_type-1.epw



-------------------- Executing FWG for sevilla_in_this_one_the_uhi_is_type-1.epw --------------------
  Full Command (for reference): java -cp "D:\OneDrive - Universidad de Cádiz (uca.es)\Programas\FutureWeatherGenerator_Europe_v1.0.1.jar" futureweathergenerator_europe.Morph D:\Python\pyfwg\pyfwg\tutorials\epws\wo_pattern\sevilla_in_this_one_the_uhi_is_type-1.epw ICHEC_EC_EARTH_SMHI_RCA4 1 0.0:0.0 72 D:\Python\pyfwg\pyfwg\tutorials\morphing_temp_results_europe\sevilla_in_this_one_the_uhi_is_type-1/ true 2 true 1 1 1:2:3
  --- FWG Real-time Output ---


2025-09-25 10:01:56 - INFO - Processing generated files in: ./morphing_temp_results_europe\sevilla_in_this_one_the_uhi_is_type-1
2025-09-25 10:01:57 - INFO - Moving './morphing_temp_results_europe\sevilla_in_this_one_the_uhi_is_type-1\ESP_-_SEVILLA_Ensemble_rcp26_2050.epw' to './final_results_workflow_europe\Seville_type-1_RCP-2.6_2050_interp-2.epw'
2025-09-25 10:01:57 - INFO - Moving './morphing_temp_results_europe\sevilla_in_this_one_the_uhi_is_type-1\ESP_-_SEVILLA_Ensemble_rcp26_2050.stat' to './final_results_workflow_europe\Seville_type-1_RCP-2.6_2050_interp-2.stat'
2025-09-25 10:01:57 - INFO - Skipping auxiliary file: 'ESP_-_SEVILLA_Ensemble_rcp26_2050_ErrorsAndWarnings.log'
2025-09-25 10:01:57 - INFO - Skipping auxiliary file: 'ESP_-_SEVILLA_Ensemble_rcp26_2050_GridPointVariables.csv'
2025-09-25 10:01:57 - INFO - Skipping auxiliary file: 'ESP_-_SEVILLA_Ensemble_rcp26_2050_MorphedEPWsComparison.csv'
2025-09-25 10:01:57 - INFO - Moving './morphing_temp_results_europe\sevilla_in_thi

  --- End of FWG Output ---


In this case, the new files we have generated are:

In [18]:
new_files_europe = [i for i in os.listdir('./final_results_workflow_europe')]
new_files_europe

['London_type-2_RCP-2.6_2050_interp-2.epw',
 'London_type-2_RCP-2.6_2050_interp-2.stat',
 'London_type-2_RCP-2.6_2080_interp-2.epw',
 'London_type-2_RCP-2.6_2080_interp-2.stat',
 'London_type-2_rcp45_2050_interp-2.epw',
 'London_type-2_rcp45_2050_interp-2.stat',
 'London_type-2_rcp45_2080_interp-2.epw',
 'London_type-2_rcp45_2080_interp-2.stat',
 'London_type-2_rcp85_2050_interp-2.epw',
 'London_type-2_rcp85_2050_interp-2.stat',
 'London_type-2_rcp85_2080_interp-2.epw',
 'London_type-2_rcp85_2080_interp-2.stat',
 'Seville_type-1_RCP-2.6_2050_interp-2.epw',
 'Seville_type-1_RCP-2.6_2050_interp-2.stat',
 'Seville_type-1_RCP-2.6_2080_interp-2.epw',
 'Seville_type-1_RCP-2.6_2080_interp-2.stat',
 'Seville_type-1_rcp45_2050_interp-2.epw',
 'Seville_type-1_rcp45_2050_interp-2.stat',
 'Seville_type-1_rcp45_2080_interp-2.epw',
 'Seville_type-1_rcp45_2080_interp-2.stat',
 'Seville_type-1_rcp85_2050_interp-2.epw',
 'Seville_type-1_rcp85_2050_interp-2.stat',
 'Seville_type-1_rcp85_2080_interp-2.ep

Let's delete the files we just generated.

In [19]:
shutil.rmtree('final_results_workflow_europe')